<a href="https://colab.research.google.com/github/steven-mpawulo/KWS_NLP_DeepLearning/blob/main/KWS_NLP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
from glob import glob
import librosa
from IPython.display import Audio
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('/content/drive/MyDrive/KWS Dataset/Train.csv')

In [3]:
df

,fn,label
0,audio_files/IV38R7F.wav,akawuka
1,audio_files/KM4SKWT.wav,banana
2,audio_files/F5POSU9.wav,obulwadde
3,audio_files/MMVDXG2.wav,nnyaanya
4,audio_files/9TVM96F.wav,pampu
...,...,...
1104,audio_files/QNZECA9.wav,cassava
1105,audio_files/GUIY718.wav,harvesting
1106,audio_files/W38UIGC.wav,farm
1107,audio_files/E04FSN7.wav,nakavundira


In [4]:
def extractmfcc(filename):
  pathToDirectory = '/content/drive/MyDrive/audio_files/'
  pathToFile = pathToDirectory+filename
  y, sr = librosa.load(pathToFile, duration=2, offset=0.5)
  z = np.mean(librosa.feature.mfcc(y=y, sr=sr,n_mfcc=40).T, axis=0)
  return z

In [5]:
X = df['fn'].apply(lambda x: extractmfcc(x))

In [6]:
X

0       [-452.45056, 50.019966, -9.084012, 6.655481, -...
1       [-104.03392, 139.45177, -7.0342426, 20.402224,...
2       [-41.928173, 90.162605, -21.265919, 28.776848,...
3       [-448.31302, 104.16875, -0.92754793, 24.056831...
4       [-366.43585, 123.214584, -42.20649, 17.503002,...
                              ...                        
1104    [-77.67435, 122.18666, -5.531857, 22.741407, 1...
1105    [-416.9687, 46.074955, 7.73032, 23.920544, -25...
1106    [-438.97623, 123.42303, 23.973106, 28.107801, ...
1107    [-632.4703, 105.0045, 7.2317038, 52.5361, -11....
1108    [-354.9069, 95.87982, -19.077534, 35.585175, 6...
Name: fn, Length: 1109, dtype: object

In [7]:
X = [x for x in X]
X = np.array(X)
X.shape

(1109, 40)

In [8]:
X = np.expand_dims(X, -1)

In [9]:
X.shape

(1109, 40, 1)

In [10]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [11]:
y = y.toarray()

In [12]:
y.shape

(1109, 193)

In [25]:
from keras import models
from keras import layers


In [27]:
model = models.Sequential()
model.add(layers.LSTM(128, return_sequences=False, input_shape=(40, 1)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(193, activation='softmax'))

model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               66560     
                                                                 
 dense_15 (Dense)            (None, 32)                4128      
                                                                 
 dropout_12 (Dropout)        (None, 32)                0         
                                                                 
 dense_16 (Dense)            (None, 64)                2112      
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dropout_14 (Dropout)        (None, 128)              

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [31]:
Epochs = 50
model.fit(X, y, validation_split=0.2, batch_size=32, shuffle=True, epochs=Epochs)

Epoch 1/100
28/28 [==============================] - 0s 9ms/step - loss: 1.3251 - accuracy: 0.5671 - val_loss: 13.2766 - val_accuracy: 0.0315
Epoch 2/100
28/28 [==============================] - 0s 10ms/step - loss: 1.1706 - accuracy: 0.6223 - val_loss: 13.3050 - val_accuracy: 0.0450
Epoch 3/100
28/28 [==============================] - 0s 10ms/step - loss: 1.2720 - accuracy: 0.5975 - val_loss: 12.8349 - val_accuracy: 0.0360
Epoch 4/100
28/28 [==============================] - 0s 8ms/step - loss: 1.2145 - accuracy: 0.6178 - val_loss: 13.2603 - val_accuracy: 0.0270
Epoch 5/100
28/28 [==============================] - 0s 9ms/step - loss: 1.1871 - accuracy: 0.6415 - val_loss: 13.8734 - val_accuracy: 0.0180
Epoch 6/100
28/28 [==============================] - 0s 9ms/step - loss: 1.1822 - accuracy: 0.6156 - val_loss: 13.7832 - val_accuracy: 0.0315
Epoch 7/100
28/28 [==============================] - 0s 10ms/step - loss: 1.2769 - accuracy: 0.6110 - val_loss: 13.4640 - val_accuracy: 0.0315
Epo